In [47]:
%load_ext autoreload
%autoreload 2

import torch
import re
import pandas as pd
import numpy as np

import operator 
from src.features.preprocess import fix_laughing_words, fix_misspelling_word, remove_punctuation, remove_url, fix_emoji, preprocess
from src.features.vectorizer import Word2vec
from torch.utils.data import Dataset, DataLoader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import gensim.downloader as api

embeddings_index = api.load("glove-wiki-gigaword-100")

In [39]:
word2vec = Word2vec(embeddings_index)

In [15]:
train_df = pd.read_csv("../data/raw/train.csv")

In [4]:
#  Remove the remain oov
def delete_oov(x, oov):
    for word in x.split():
        for vocab in oov:
            if word == vocab[0]:
                x = x.replace(word, '')
    return x

def tokenize_fn(text):
    return text.split()

In [49]:
from src.data.dataset import TextDataset

ds = TextDataset("../data/raw/train.csv",
                 preprocess_fn=preprocess,
                 tokenize_fn=tokenize_fn,
                 vectorizer_fn=word2vec.get_vector_average)

In [50]:
val_ratio = 0.1
num_val = int(val_ratio * len(ds))
num_train = len(ds) - num_val
train_ds, val_ds = torch.utils.data.random_split(ds, [num_train, num_val])
len(train_ds), len(val_ds)

(22259, 2473)

In [66]:
train_loader = DataLoader(train_ds, batch_size=1024, shuffle=False)

In [85]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(100, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x


net = Net()

In [86]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [91]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % 2000 == 1999:    # print every 2000 mini-batches
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        running_loss = 0.0

print('Finished Training')

[1,     1] loss: nan
[1,     2] loss: nan
[1,     3] loss: nan
[1,     4] loss: nan
[1,     5] loss: nan
[1,     6] loss: nan
[1,     7] loss: nan
[1,     8] loss: nan
[1,     9] loss: nan
[1,    10] loss: nan
[1,    11] loss: nan
[1,    12] loss: nan
[1,    13] loss: nan
[1,    14] loss: nan
[1,    15] loss: nan
[1,    16] loss: nan
[1,    17] loss: nan
[1,    18] loss: nan
[1,    19] loss: nan
[1,    20] loss: nan
[1,    21] loss: nan
[1,    22] loss: nan
[2,     1] loss: nan
[2,     2] loss: nan
[2,     3] loss: nan
[2,     4] loss: nan
[2,     5] loss: nan
[2,     6] loss: nan
[2,     7] loss: nan
[2,     8] loss: nan
[2,     9] loss: nan
[2,    10] loss: nan
[2,    11] loss: nan
[2,    12] loss: nan
[2,    13] loss: nan
[2,    14] loss: nan
[2,    15] loss: nan
[2,    16] loss: nan
[2,    17] loss: nan
[2,    18] loss: nan


KeyboardInterrupt: 

In [ ]:
val_loader = DataLoader(val_ds, batch_size=512, shuffle=False)
corrects = []
for batch in tqdm(val_loader):
    X_batch, y_batch = batch
    predicted = net(X_batch.float())
    print(predicted)